# 📊 Multi-Source Data Aggregation for Market Context Modeling

This notebook builds a consolidated pipeline for collecting and merging:

- **Market data** from Yahoo Finance  
- **Macroeconomic indicators** from the FRED API  
- **Sentiment signals** from Google Trends  

These sources power downstream analysis and modeling for the Pendle project.

## 📦 Setup & Imports

In [3]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath("../scripts"))

## 📈 Market Data: Yahoo Finance

Daily OHLCV data for:
- `^GSPC`, `^IXIC`, `^VIX`, `BTC-USD`  
📁 Output: `../data/raw/yahoo_market_data.csv`

In [12]:
from fetch_yahoo import fetch_market_data

# ⚠️ UNCOMMENT TO RUN ONCE
# market_df = fetch_market_data(start="2000-01-01", save_path="../data/raw/yahoo_market_data.csv")

# ✅ Load from cache
market_df = pd.read_csv("../data/raw/yahoo_market_data.csv", index_col=0, parse_dates=True)
print("📅 Yahoo Finance Date Range:", market_df.index.min(), "→", market_df.index.max())
market_df.head()

📅 Yahoo Finance Date Range: 2000-01-03 00:00:00 → 2025-04-14 00:00:00


,Open_^GSPC,High_^GSPC,Low_^GSPC,Close_^GSPC,Adj Close_^GSPC,Volume_^GSPC,Open_BTC-USD,High_BTC-USD,Low_BTC-USD,Close_BTC-USD,...,Low_^VIX,Close_^VIX,Adj Close_^VIX,Volume_^VIX,Open_^IXIC,High_^IXIC,Low_^IXIC,Close_^IXIC,Adj Close_^IXIC,Volume_^IXIC
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,1455.219971,9.318000e+08,NaN,NaN,NaN,NaN,...,23.980000,24.209999,24.209999,0.0,4186.189941,4192.189941,3989.709961,4131.149902,4131.149902,1.510070e+09
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1399.420044,1.009000e+09,NaN,NaN,NaN,NaN,...,24.799999,27.010000,27.010000,0.0,4020.000000,4073.250000,3898.229980,3901.689941,3901.689941,1.511840e+09
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1402.109985,1.085500e+09,NaN,NaN,NaN,NaN,...,25.850000,26.410000,26.410000,0.0,3854.350098,3924.209961,3734.870117,3877.540039,3877.540039,1.735670e+09
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1403.449951,1.092300e+09,NaN,NaN,NaN,NaN,...,24.700001,25.730000,25.730000,0.0,3834.439941,3868.760010,3715.620117,3727.129883,3727.129883,1.598320e+09
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1441.469971,1.225200e+09,NaN,NaN,NaN,NaN,...,21.719999,21.719999,21.719999,0.0,3711.090088,3882.669922,3711.090088,3882.620117,3882.620117,1.634930e+09


## 📊 Macroeconomic Indicators: FRED

Selected indicators:
- Bond Yields (`DGS10`)
- Inflation (`CPIAUCSL`)
- Unemployment (`UNRATE`)
- Interest Rates (`FEDFUNDS`)
- Consumer Sentiment (`UMCSENT`)
- GDP (`GDP`)  
📁 Output: `../data/raw/fred_macro_data.csv`

In [16]:
from fetch_fred import fetch_fred_series

fred_series = {
    "Bond Yields": "DGS10",
    "Inflation": "CPIAUCSL",
    "Unemployment": "UNRATE",
    "Interest Rate": "FEDFUNDS",
    "Consumer Sentiment": "UMCSENT",
    "GDP": "GDP"
}

# ⚠️ UNCOMMENT TO RUN ONCE
# macro_df = fetch_fred_series(fred_series, start="2000-01-01", save_path="../data/raw/fred_macro_data.csv")

# ✅ Load from cache
macro_df = pd.read_csv("../data/raw/fred_macro_data.csv", index_col=0, parse_dates=True)
print("📅 FRED Date Range:", macro_df.index.min(), "→", macro_df.index.max())
macro_df.head()

📅 FRED Date Range: 2000-01-03 00:00:00 → 2025-04-14 00:00:00


,Bond Yields,Inflation,Unemployment,Interest Rate,Consumer Sentiment,GDP
2000-01-03,6.58,NaN,NaN,NaN,NaN,NaN
2000-01-04,6.49,NaN,NaN,NaN,NaN,NaN
2000-01-05,6.62,NaN,NaN,NaN,NaN,NaN
2000-01-06,6.57,NaN,NaN,NaN,NaN,NaN
2000-01-07,6.52,NaN,NaN,NaN,NaN,NaN


## 🧠 Sentiment Signals: Google Trends

Terms:
- `"market crash"`, `"recession"`, `"buy gold"`, `"stock market crash"`  
📁 Output: Chunked into 5 raw files → merged to `../data/processed/google_trends_full.csv`

In [18]:
from fetch_sentiment import fetch_google_trends_by_year
import json

terms = ["market crash", "recession", "buy gold", "stock market crash"]

### 🔄 Chunked Pulls (UNCOMMENT to fetch)

In [ ]:
# 🚀 Run Google Trends Sentiment Pull — Chunk 1 (2004–2007)
# This function slices each keyword by **year** to allow daily-resolution pulls
# while staying under Google’s rate limits. It is deliberately slow and cautious.

# 🧠 Sentiment-related search terms
# - "market crash", "recession", "buy gold", "stock market crash"
# - These represent fear/flight-to-safety behaviors in investor psychology.

# ⚠️ UNCOMMENT TO DEPLOY THIS MISSION ONLY ONCE — pulls daily resolution by year
# Chunk 1: 2004–2007
# Pulls Google Trends data in ultra-safe mode — by keyword, by year — with max stealth cooldowns
# Useful for behavioral signals like fear, recession concern, and flight-to-safety instincts
sentiment_df_1 = fetch_google_trends_by_year(
    keywords=terms,                                       # 🎯 Target terms: market panic + hedge signals
    start_year=2004,                                      # 🗓️ Start of available Google Trends data
    end_year=2007,                                        # ⏳ Cutoff for this batch
    save_path="../data/raw/google_trends_2004_2007.csv",  # 💾 Drop point for raw intel
    batch_size=1,                                         # 🚨 Solo keyword pulls — avoid 400 Bad Requests
    cooldown=74                                           # 🧊 MAX STEALTH: long sleep to dodge throttling (429 errors)
)

In [24]:
# Chunk 2: 2008–2011
sentiment_df_2 = fetch_google_trends_by_year(
    keywords=terms,
    start_year=2008,
    end_year=2011,
    save_path="../data/raw/google_trends_2008_2011.csv",
    batch_size=1,
    cooldown=74
)

📈 Fetching Google Trends data (daily granularity) for: ['market crash', 'recession', 'buy gold', 'stock market crash']
🔁 Year 2008, Batch 1: ['market crash']
🔁 Year 2008, Batch 2: ['recession']
❌ Error fetching ['recession'] for 2008: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=en-US&tz=360&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22recession%22%2C+%22time%22%3A+%222008-01-01+2008-12-31%22%2C+%22geo%22%3A+%22US%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1472fe9d0>, 'Connection to trends.google.com timed out. (connect timeout=2)'))
⏳ Cooling down before retrying...
🔁 Year 2008, Batch 3: ['buy gold']
🔁 Year 2008, Batch 4: ['stock market crash']
❌ Error fetching ['stock market crash'] for 2008: The request failed: Google returned a response with code 429
⏳ Cooling down before retrying...
🔁 Year 2009, Batch 1: ['

In [30]:
# 🕵️ Sentiment Reconnaissance: Chunk 3 (2012–2015)
# Pulls Google Trends data in max stealth mode — daily resolution, 1 keyword at a time.
# Saves any failed (keyword, year) pairs to a .json file for retry later.

sentiment_df_3, failures_3 = fetch_google_trends_by_year(
    keywords=terms,
    start_year=2012,
    end_year=2015,
    save_path="../data/raw/google_trends_2012_2015.csv",
    batch_size=1,
    cooldown=6
)

# 💾 Save failed (keyword, year) pairs for retry
with open("../logs/failures_2012_2015.json", "w") as f:
    json.dump(failures_3, f)

print("📦 Saved Chunk 3")
print("🧯 Failed fetch attempts:", failures_3)

📈 Fetching Google Trends data (daily granularity) for: ['market crash', 'recession', 'buy gold', 'stock market crash']
🔁 Year 2012, Batch 1: ['market crash']
❌ Error fetching ['market crash'] for 2012: The request failed: Google returned a response with code 429
⏳ Cooling down before retrying...
🔁 Year 2012, Batch 2: ['recession']
❌ Error fetching ['recession'] for 2012: HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
⏳ Cooling down before retrying...
🔁 Year 2012, Batch 3: ['buy gold']
❌ Error fetching ['buy gold'] for 2012: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=en-US&tz=360&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22buy+gold%22%2C+%22time%22%3A+%222012-01-01+2012-12-31%22%2C+%22geo%22%3A+%22US%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x14ea85fd0>, 'Connection t

ValueError: not enough values to unpack (expected 2, got 1)

In [34]:
# Chunk 4: 2016–2019
sentiment_df_4 = fetch_google_trends_by_year(
    keywords=terms,
    start_year=2016,
    end_year=2019,
    save_path="../data/raw/google_trends_2016_2019.csv",
    batch_size=1,
    cooldown=74
)

📈 Fetching Google Trends data (daily granularity) for: ['market crash', 'recession', 'buy gold', 'stock market crash']
🔁 Year 2016, Batch 1: ['market crash']
❌ Error fetching ['market crash'] for 2016: The request failed: Google returned a response with code 429
⏳ Cooling down before retrying...
🔁 Year 2016, Batch 2: ['recession']
❌ Error fetching ['recession'] for 2016: The request failed: Google returned a response with code 429
⏳ Cooling down before retrying...
🔁 Year 2016, Batch 3: ['buy gold']
❌ Error fetching ['buy gold'] for 2016: The request failed: Google returned a response with code 429
⏳ Cooling down before retrying...
🔁 Year 2016, Batch 4: ['stock market crash']
❌ Error fetching ['stock market crash'] for 2016: The request failed: Google returned a response with code 429
⏳ Cooling down before retrying...
🔁 Year 2017, Batch 1: ['market crash']
❌ Error fetching ['market crash'] for 2017: The request failed: Google returned a response with code 429
⏳ Cooling down before retry

In [ ]:
# Chunk 5: 2020–2025
sentiment_df_5 = fetch_google_trends_by_year(
    keywords=terms,
    start_year=2020,
    end_year=2025,
    save_path="../data/raw/google_trends_2020_2025.csv",
    batch_size=1,
    cooldown=74
)

In [ ]:
# 📝 Define missing keyword-year pairs manually
failures_1 = [
    ("recession", 2006)
]

In [22]:
sentiment_df_1 = fetch_google_trends_by_year(
    keywords=terms,                                       # ✅ Your list of search terms
    start_year=2004,                                      # ✅ First year to pull
    end_year=2007,                                        # ✅ Last year to pull
    save_path="../data/raw/google_trends_2004_2007.csv",  # ✅ Output location
    batch_size=1,                                         # ✅ Pull one keyword at a time to prevent 400s
    cooldown=74                                           # ✅ Sleep to avoid throttling, ultra-conservative max stealth
)

📈 Fetching Google Trends data (daily granularity) for: ['market crash', 'recession', 'buy gold', 'stock market crash']
🔁 Year 2004, Batch 1: ['market crash']
🔁 Year 2004, Batch 2: ['recession']
🔁 Year 2004, Batch 3: ['buy gold']
🔁 Year 2004, Batch 4: ['stock market crash']
🔁 Year 2005, Batch 1: ['market crash']
🔁 Year 2005, Batch 2: ['recession']
🔁 Year 2005, Batch 3: ['buy gold']
🔁 Year 2005, Batch 4: ['stock market crash']
🔁 Year 2006, Batch 1: ['market crash']
🔁 Year 2006, Batch 2: ['recession']
❌ Error fetching ['recession'] for 2006: The request failed: Google returned a response with code 429
⏳ Cooling down before retrying...
🔁 Year 2006, Batch 3: ['buy gold']
🔁 Year 2006, Batch 4: ['stock market crash']
🔁 Year 2007, Batch 1: ['market crash']
🔁 Year 2007, Batch 2: ['recession']
🔁 Year 2007, Batch 3: ['buy gold']
🔁 Year 2007, Batch 4: ['stock market crash']
✅ Saved Google Trends data to ../data/raw/google_trends_2004_2007.csv


In [ ]:
# sentiment_df_2 = fetch_google_trends_by_year(terms, 2008, 2011, "../data/raw/google_trends_2008_2011.csv", 1, 6)

In [ ]:
# sentiment_df_3 = fetch_google_trends_by_year(terms, 2012, 2015, "../data/raw/google_trends_2012_2015.csv", 1, 6)

In [ ]:
# sentiment_df_4 = fetch_google_trends_by_year(terms, 2016, 2019, "../data/raw/google_trends_2016_2019.csv", 1, 6)

In [ ]:
# sentiment_df_5 = fetch_google_trends_by_year(terms, 2020, 2025, "../data/raw/google_trends_2020_2025.csv", 1, 6)

### ✅ Load and Merge Sentiment Chunks

In [7]:
sentiment_chunks = ["2004_2007", "2008_2011", "2012_2015", "2016_2019", "2020_2025"]

dfs = []
for span in sentiment_chunks:
    df = pd.read_csv(f"../data/raw/google_trends_{span}.csv", index_col=0, parse_dates=True)
    print(f"📅 Trends Chunk {span}:", df.index.min(), "→", df.index.max())
    dfs.append(df)

sentiment_df = pd.concat(dfs).sort_index()
sentiment_df.to_csv("../data/processed/google_trends_full.csv")
print("✅ Final merged sentiment dataset saved.")

📅 Trends Chunk 2004_2007: nan → nan


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/google_trends_2008_2011.csv'

## 📦 Final Merge: Market + Macro + Sentiment

In [ ]:
# 📥 Load all cleaned datasets
market_df = pd.read_csv("../data/raw/yahoo_market_data.csv", index_col=0, parse_dates=True)
macro_df = pd.read_csv("../data/raw/fred_macro_data.csv", index_col=0, parse_dates=True)
sentiment_df = pd.read_csv("../data/processed/google_trends_full.csv", index_col=0, parse_dates=True)

# 🔗 Merge all sources
merged = market_df.join(macro_df, how="outer").join(sentiment_df, how="outer")
merged = merged.sort_index()

# 💾 Save full merged dataset
merged.to_csv("../data/processed/merged_all_sources.csv")

# 👀 Preview merged output
print("📊 Final Merged Dataset:")
print("Date Range:", merged.index.min(), "→", merged.index.max())
merged.head()

## ✅ Summary
We successfully built a modular and interpretable ingestion pipeline for:

- ✅ Daily market data (`yfinance`)
- ✅ Macroeconomic indicators (`FRED`)
- ✅ Sentiment trends (`Google Trends` via `pytrends`)

📁 All data has been cached, aligned, and merged into a single file:
```
../data/processed/merged_all_sources.csv
```

Next steps: **EDA, lag feature engineering, and predictive modeling.**